In [1]:
import numpy as np
import pandas as pd
import collections

In [12]:
def translate(seq): 
    seq = seq.upper()
    seq_cut = seq[:len(seq)-len(seq)%3]
    table = { 
        'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M', 
        'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T', 
        'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K', 
        'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',                  
        'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L', 
        'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P', 
        'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q', 
        'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R', 
        'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V', 
        'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A', 
        'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E', 
        'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G', 
        'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S', 
        'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L', 
        'TAC':'Y', 'TAT':'Y', 'TAA':'_', 'TAG':'_', 
        'TGC':'C', 'TGT':'C', 'TGA':'_', 'TGG':'W', 
    }  
    protein ="" 
    for i in range(0, len(seq_cut), 3): 
        codon = seq_cut[i:i + 3] 
        protein+= table[codon]
    if len(seq)%3 != 0:
        protein+="-"
    #print(protein.lower())
    return protein.upper()

In [58]:
seq = "agtaGTttgAGcgtg"
mut_seq="agtaGAGtgAGcgtg"

In [65]:
translate(seq)

'sslsv'

In [66]:
translate(mut_seq)

'srvsv'

In [91]:
boolean = []
for i in range(len(translate(seq))):
    if (translate(seq)[i] == translate(mut_seq)[i]):
        boolean.append('T')
    else:
        boolean.append('F')

In [107]:
group = [k for k in range(len(boolean)) if boolean[k]=='T']
group

[0, 3, 4]

In [175]:
def score_codon_mismatch(codon1, codon2):
    score = 0
    for i in range(len(codon1)):
        if codon1[i] != codon2[i]:
            score +=1
    return score
#codon table deleted of rare codons in E coli
table = { 
        'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M', 
        'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T', 
        'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K', 
        'AGC':'S', 'AGT':'S',                 
        'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L', 
        'CCA':'P',  'CCG':'P', 'CCT':'P', 
        'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q', 
             'CGC':'R', 'CGT':'R', 
        'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V', 
        'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A', 
        'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E', 
        'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G', 
        'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S', 
        'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L', 
        'TAC':'Y', 'TAT':'Y', 'TAA':'_', 'TAG':'_', 
        'TGC':'C', 'TGT':'C', 'TGA':'_', 'TGG':'W', 
    }
def nu_favored_replacement(codon, mut_codon,table=table):
    codon = codon.upper()
    mut_codon = mut_codon.upper()
    codon_list = []
    for k,v in table.items():
        codon_list.append([k,v]) 
    codon_list = np.array(codon_list)
    #print(codon_list)
    counter = collections.Counter(codon_list[:,1])
    replicates = np.array([k for k in counter.values()])
    aa = np.array([k for k in counter.keys()])
    #print(codon_list[:,1])
    if (codon in codon_list[:,0]) and (mut_codon in codon_list[:,0]):
        aa_codon = table[codon]
        #print(aa_codon)
        aa_mut_codon = table[mut_codon]
        #aa_where = [i for i in range(len(aa)) if aa_codon==aa[i]]
        codons = np.array([codon_list[i,0] for i in range(codon_list.shape[0]) if codon_list[i,1]==aa_codon])
        #print(codons)
        mut_codons = np.array([codon_list[i,0] for i in range(codon_list.shape[0]) if codon_list[i,1]==aa_mut_codon])
        dim = codons.shape[0]*mut_codons.shape[0]
        matching_scores = np.zeros((dim))
        matching_pairs = np.empty((dim,2), dtype='>U10')
        count = 0
        for i in range(codons.shape[0]):
            #print (codons[i])
            for j in range(mut_codons.shape[0]):
                matching_scores[count] = score_codon_mismatch(codons[i],mut_codons[j])
                matching_pairs[count] = np.array([codons[i],mut_codons[j]])
                count +=1
        min_match = np.argmin(matching_scores)
        #print(matching_pairs)
        #print(codons[5])
        if matching_pairs[min_match,0] == codon:
            new_codon = codon.lower()
        else:
            new_codon = matching_pairs[min_match,0].upper()
        #print(new_codon)
        #print(matching_pairs)
    else:
        new_codon = codon.lower()
    #print(new_codon)
    return new_codon
            
    
    
def renew_codon(seq, mut_seq):
    seq = seq.lower()
    mut_seq = mut_seq.lower()
    protein = translate(seq)
    mut_protein = translate(mut_seq)
    aa_detect = []
    for i in range(len(protein)):
        if (protein[i] == mut_protein[i]):
            aa_detect.append('T')
        else:
            aa_detect.append('F')
    aa_detect_index = [k for k in range(len(aa_detect)) if aa_detect[k]=='F']
    for k in aa_detect_index:
        new_codon = list(nu_favored_replacement(seq[3*k:3*k+3],mut_seq[3*k:3*k+3]))
        #codon_replaced = seq[3*k:3*k+3]
        seq_list = list(seq)
        seq_list[3*k:3*k+3] = new_codon
        seq = ''.join(seq_list)
        #codon_replaced.replace(codon_replaced, new_codon)
        #print(codon_replaced)
    return seq
    
def renew_codon_from_mut_protein(seq, mut_protein,table=table):
    seq = seq.lower()
    mut_protein = mut_protein.upper()
    protein = translate(seq)
    aa_detect=[]
    for i in range(len(protein)):
        if (protein[i] == mut_protein[i]):
            aa_detect.append('T')
        else:
            aa_detect.append('F')
    #print(aa_detect)
    aa_detect_index = np.array([k for k in range(len(aa_detect)) if aa_detect[k]=='F'])
    codon_list = []
    for k,v in table.items():
        codon_list.append([k,v]) 
    codon_list = np.array(codon_list)
    #for mut_point in aa_detect_index:
    #print(aa_detect_index)
    for mut_point in aa_detect_index:
        mut_codons = np.array([codon_list[i,0] for i in range(codon_list.shape[0]) if codon_list[i,1]==mut_protein[mut_point]])
    #print(mut_codons)
        matching_scores = np.zeros((mut_codons.shape[0]))  
        #print(mut_codons.shape[0])
        #print(aa_detect_index.shape)
        #print(mut_codons.shape)
        new_codons = np.empty((mut_codons.shape[0]), dtype='<U10')
        count = 0
        
        for mut_codon in mut_codons:
            #print(mut_codon)
            codon_replaced = seq[3*mut_point:3*mut_point+3]
            new_codon = nu_favored_replacement(codon_replaced,mut_codon)
            matching_scores[count]=score_codon_mismatch(new_codon,mut_codon)
            new_codons[count] = new_codon
            count+=1
            #print(new_codons)
        min_score = np.argmin(matching_scores)
            #print(matching_scores)
            #print(min_score)
        new_codon = new_codons[min_score]
    #check this
        #print(new_codons)
        new_codon = list(new_codon)
        #print(new_codon)
        seq_list = list(seq)
        seq_list[3*mut_point:3*mut_point+3] = new_codon
        #print(new_codon)
        seq = ''.join(seq_list)
            #count+=1
        #seq = seq.replace(codon_replaced, new_codons[min_score])
    return seq

In [176]:
seq = "atgagtatgttggtcgtggtcactgaaaatgtacctccgcgcttacgaggcagattagccatctggttgttggaggtacgtgcaggggtatatgtaggtgatgtatccgcaaaaattcgtgaaatgatctgggaacaaatagctggactggcggaagaaggcaatgtagtgatggcatgggcaacgaatacggaaacgggatttgagttccagacatttgggttaaacagg"
mut_seq = "atgagtaGTttgAGcgtgTAcactgaaaatgtacctccgcgcttacgaggcagattagccatctggttgttggagCtacgtgcagggTACtatCtaggtgatgtatccgcaaaaattcgtgaaatgatctgggaacaaatagctggactggcggaagaaggcaatgtaCtgatgTcatggTcaacgaatacggaaacgggattAgagttGcagacatttgggttaaacagg"
mut_protein = "MSLLEVSTENVPPRLRGRLAIWLVEFRAGSYIGDVSAKIREMIWEQIAGLAEEGNVVMNWYTNTETGFEYQTYGLNQ"
#renew_codon(seq,mut_seq)
renew_codon_from_mut_protein(seq,mut_protein)

'atgagtatgttgGTAgtgGTAactgaaaatgtacctccgcgcttacgaggcagattagccatctggttgCTAgagGTCcgtgcaggggtatatgtaggtgatgtatccgcaaaaattcgtgaaatgatctgggaacaaatagctggactggcggaagaaggcaatgtagtgatgGCCtggGCCacgaatacggaaacgggatttgagttccagacaTTCgggttaaacagg'

In [83]:
translate(seq)

'MSMLVVVTENVPPRLRGRLAIWLLEVRAGVYVGDVSAKIREMIWEQIAGLAEEGNVVMAWATNTETGFEFQTFGLNR'

In [155]:
[codon_list[i,0] for i in range(codon_list.shape[0]) if ]

ValueError: 'Y' is not in list

In [184]:

[[k,v] for k,v in counter]

ValueError: not enough values to unpack (expected 2, got 1)

In [187]:
counter.keys()

dict_keys(['I', 'M', 'T', 'N', 'K', 'S', 'R', 'L', 'P', 'H', 'Q', 'V', 'A', 'D', 'E', 'G', 'F', 'Y', '_', 'C', 'W'])